In [58]:
!pip install node2vec
!pip install optuna


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import networkx as nx
from node2vec import Node2Vec
from igraph import Graph
import igraph as ig
import numpy as np
import json
import pandas as pd
from gensim.models import Word2Vec
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, log_loss, matthews_corrcoef, precision_recall_curve, average_precision_score
from sklearn.metrics import precision_recall_fscore_support as prf_support
from sklearn.metrics import roc_curve, roc_auc_score, auc
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


In [4]:
graph = r'/Users/georgeilli/Desktop/NeurIPS/NeurIPS-2024-GANTAT/Network Construction/less_Tclin_Signalink_PIN_graph.graphml'
# Create an igraph object
PPI_graph = ig.Graph.Load(graph, format='graphml')
# Set node names for igraph graph

PPI_graphx = Graph.to_networkx(PPI_graph)

In [7]:
# Specify the parameters for node2vec
dimensions= 150
walk_length=65
num_walks= 225
p=1
q=0.2

# Create a node2vec model
node2vec = Node2Vec(PPI_graphx, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=10, p=p,q=q, seed=42)

# Fit the model to generate embeddings
model = node2vec.fit(window=10, min_count=1, batch_words=4)

embedding_data = []
for node in model.wv.key_to_index:
    vector = model.wv.get_vector(node)
    node_name = PPI_graphx.nodes[int(node)]['name']  # Retrieve node name from the graph
    embedding_data.append([node_name] + vector.tolist())

column_names = ['node_name'] + [f'dim_{i}' for i in range(model.vector_size)]
embedding_df = pd.DataFrame(embedding_data, columns=column_names)

'''
Combination 4:

dimensions: 100
walk_length: 50
num_walks: 300
p: 0.7
q: 1.8

Accuracy for Tclin: 0.9310
AUC for Tclin: 0.7189
AUPR for Tclin: 0.3466
MCC for Tclin: 0.2982
F1 Score for Tclin: 0.2500

Combination 5:

dimensions: 100
walk_length: 40
num_walks: 250
p: 0.3
q: 2.5

Accuracy for Tclin: 0.9425
AUC for Tclin: 0.7467
AUPR for Tclin: 0.3838
MCC for Tclin: 0.3963
F1 Score for Tclin: 0.3137

Combination 6:

dimensions: 100
walk_length: 60
num_walks: 200
p: 0.5
q: 2

Accuracy for Tclin: 0.9327
AUC for Tclin: 0.7522
AUPR for Tclin: 0.3622
MCC for Tclin: 0.3569
F1 Score for Tclin: 0.2679

'''

Computing transition probabilities:   0%|          | 0/6086 [00:00<?, ?it/s]

Generating walks (CPU: 10): 100%|██████████| 22/22 [00:03<00:00,  6.07it/s]


'\nCombination 4:\n\ndimensions: 100\nwalk_length: 50\nnum_walks: 300\np: 0.7\nq: 1.8\n\nAccuracy for Tclin: 0.9310\nAUC for Tclin: 0.7189\nAUPR for Tclin: 0.3466\nMCC for Tclin: 0.2982\nF1 Score for Tclin: 0.2500\n\nCombination 5:\n\ndimensions: 100\nwalk_length: 40\nnum_walks: 250\np: 0.3\nq: 2.5\n\nAccuracy for Tclin: 0.9425\nAUC for Tclin: 0.7467\nAUPR for Tclin: 0.3838\nMCC for Tclin: 0.3963\nF1 Score for Tclin: 0.3137\n\nCombination 6:\n\ndimensions: 100\nwalk_length: 60\nnum_walks: 200\np: 0.5\nq: 2\n\nAccuracy for Tclin: 0.9327\nAUC for Tclin: 0.7522\nAUPR for Tclin: 0.3622\nMCC for Tclin: 0.3569\nF1 Score for Tclin: 0.2679\n\n'

In [77]:
embedding_df

,node_name,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,...,dim_140,dim_141,dim_142,dim_143,dim_144,dim_145,dim_146,dim_147,dim_148,dim_149
0,MYB,-0.216255,-0.107645,-0.014307,-0.438286,-0.578351,0.146458,0.067725,0.535546,-0.599110,...,1.130146,0.370006,0.471096,0.363378,0.940983,0.556338,-0.020605,0.564439,-0.164202,1.136991
1,PRF1,0.551127,0.233373,-0.196633,0.151449,0.000748,-0.168720,-0.177596,0.770812,-0.858456,...,0.215534,0.353621,0.876463,-0.223144,1.086182,0.303030,0.178223,0.388203,0.351607,-0.998019
2,DAAM1,0.272970,0.961753,-0.428428,-0.294270,0.303346,-0.132765,-0.134651,0.426102,0.798383,...,0.720128,-0.604653,0.499548,1.051346,0.772144,-0.687301,0.485578,0.419102,-0.012441,-0.198391
3,RNASEL,0.594605,0.389114,-0.055574,-0.472768,-0.140309,-0.313714,-0.639050,-0.057030,-0.135569,...,0.285586,0.101241,0.959722,-0.045092,0.738960,0.525800,1.184398,0.439001,-0.017465,-0.333974
4,AIM2,0.489497,0.570536,-0.215613,-0.371642,-0.031233,-0.503305,-0.031406,0.331452,-0.223851,...,0.931313,-0.555517,-0.042889,-0.193921,1.127598,0.119547,0.005078,0.766907,-0.346295,-0.951418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,RIPK4,-0.249458,0.456244,0.410649,-0.085891,-0.052768,-0.472054,-0.450054,-0.381906,0.386411,...,0.208579,-0.593820,0.501652,0.162404,0.129175,-0.220009,-0.493010,0.393965,-0.104724,-0.288055
6082,DDR2,0.124040,0.082359,-0.101277,0.049203,-0.167279,-0.052479,0.348018,0.220322,0.319195,...,0.174748,0.181998,0.297964,0.151150,0.384046,-0.146692,-0.223180,0.128156,0.207164,0.000912
6083,ABHD2,-0.129421,0.124130,-0.439750,0.266506,-0.239258,-0.166710,0.110963,0.199853,-0.049171,...,-0.121820,-0.165298,0.073458,0.119613,-0.355111,-0.398630,0.471072,-0.215748,-0.155136,-0.257492
6084,BTN3A1,-0.173268,0.133313,-0.351877,0.159892,-0.078821,0.227840,0.327381,0.575049,-0.268209,...,0.289387,-0.570576,0.377945,0.408889,-0.352312,-0.102619,-0.066693,-0.239838,-0.254465,-0.696142


In [26]:
def split_balance(df, n_splits=20, random_state=42, balance_ratio=1):
    np.random.seed(random_state)
    positive_samples = df[df['label'] == 1]
    negative_samples = df[df['label'] == 0]
    
    
    folds = []
    
    for _ in range(n_splits):
        # Select 80% of positive samples
        pos_fold = positive_samples.sample(frac=0.8, random_state=random_state)
        
        # Calculate the number of negative samples needed
        num_neg_samples = int(len(pos_fold) * balance_ratio)
        
        # Randomly select negative samples
        neg_fold = negative_samples.sample(n=num_neg_samples, random_state=random_state)
        
        # Combine positive and negative samples
        fold = pd.concat([pos_fold, neg_fold])
        
        # Shuffle the fold
        fold = fold.sample(frac=1, random_state=random_state).reset_index(drop=True)
        
        folds.append(fold)
    
    return folds

def aupr_score(y_true, y_pred_proba):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    return auc(recall, precision)

def compute_metrics(y_true, y_proba, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_proba),
        'aupr': aupr_score(y_true, y_proba)
    }


def custom_cross_validate(X, hype=None, n_splits=5, spliting=20, random_state=42, balance_ratio=1, classifier='XgBoost'):
    # Initialize KFold cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    all_metrics = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        
        x_df_train = pd.DataFrame(X_train)
        x_df_test = pd.DataFrame(X_test)

        folds = split_balance(x_df_train, n_splits=spliting, balance_ratio=balance_ratio)
        models = train(folds, hype=hype, classifier=classifier)
        pre_proba = evaluate(x_df_test, models)
        
        y_pred = (pre_proba >= 0.5).astype(int)
        
        metrics = compute_metrics(x_df_test['label'], pre_proba, y_pred)
        all_metrics.append(metrics)

    return all_metrics

def train(folds, hype=None, classifier='XgBoost'):

    models = []

    for i in range(len(folds)):

        X_features = folds[i].drop(['Gene_name'], axis=1)
        y = X_features['label']
        X_features = X_features.drop('label', axis=1)

        if classifier=='XgBoost':
            # Define the XGBoost model
            if hype != None:
                model = xgb.XGBClassifier(**hype)
                model.fit(X_features, y)
            else:
                model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
                model.fit(X_features, y)
                
        elif classifier == 'DT':
            
            if hype is not None:
                model = DecisionTreeClassifier(**hype)
            else:
                model = DecisionTreeClassifier(random_state=42)
            model.fit(X_features, y)

        elif classifier == 'RF':
            
            # Define the Random Forest model
            if hype is not None:
                model = RandomForestClassifier(**hype)
            else:
                model = RandomForestClassifier(random_state=42)
            model.fit(X_features, y)

        models.append(model)

    return models

def evaluate(df_test, models):

    test_features = df_test.drop(['Gene_name'], axis=1)
    y = test_features['label']
    test_features = test_features.drop('label', axis=1)
    predictions_proba = []

    
    for model in models:
        # Predict probabilities
        proba = model.predict_proba(test_features)[:, 1]  # Probabilities for the positive class
        predictions_proba.append(proba)

    predictions_proba = np.array(predictions_proba)
    avg_probas = np.mean(predictions_proba, axis=0)

    return avg_probas

def compute_mean_and_std_metrics(metrics):
    # Initialize dictionaries to hold the sums and squared sums of each metric
    metric_sums = {key: 0 for key in metrics[0].keys()}
    metric_squared_sums = {key: 0 for key in metrics[0].keys()}

    # Sum each metric and squared metric across all folds
    for metric in metrics:
        for key, value in metric.items():
            metric_sums[key] += value
            metric_squared_sums[key] += value ** 2

    n = len(metrics)
    # Calculate the mean for each metric
    metric_means = {key: value / n for key, value in metric_sums.items()}

    # Calculate the standard deviation for each metric
    metric_stds = {
        key: ((metric_squared_sums[key] / n) - (metric_means[key] ** 2)) ** 0.5 for key in metrics[0].keys()
    }

    # Construct the string with mean ± std for each metric
    metric_results = {key: f"{metric_means[key]:.4f} ± {metric_stds[key]:.4f}" for key in metrics[0].keys()}

    return metric_results

def pipeline(X, pos_y, hype=None, cross_split=5, balance_split=30, balance_ratio=1, classifier='XgBoost'):

    positive_labels = pd.read_csv(pos_y, header=None)
    X['label'] = X['Gene_name'].isin(positive_labels[1]).astype(int)
    result = custom_cross_validate(X, hype=hype, n_splits=cross_split, spliting=balance_split, balance_ratio=balance_ratio, classifier=classifier)
    met = compute_mean_and_std_metrics(result)

    for keys, value in met.items():
        print(keys + ': ' + value)

    return met

def grid_search(X, pos_y, param_grid, cross_split=5, balance_split=30, balance_ratio=1, classifier='XgBoost'):
    # Generate all combinations of hyperparameters
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    results = []
    trial = 0
    for params in combinations:
        trial +=1
        # Update the hype dictionary with the current combination of hyperparameters

        '''
        hype = {
            'learning_rate': params['learning_rate'],  # Different learning rates
            'max_depth': params['max_depth'],               # Different depths of trees
            'n_estimators': params['n_estimators'],           # Number of trees
            'subsample': params['subsample'],             # Fraction of samples used for fitting
            'colsample_bytree': params['colsample_bytree'] # Whether bootstrap samples are used when building trees
        }
        
        hype = {
            'n_estimators': params['n_estimators'],            # Number of trees in the forest
            'max_depth': params['max_depth'],            # Maximum depth of the tree
            'min_samples_split': params['min_samples_split'],           # Minimum number of samples required to split an internal node
            'min_samples_leaf': params['min_samples_leaf'],             # Minimum number of samples required to be at a leaf node
            'max_features': params['max_features'],    # Number of features to consider when looking for the best split
            'bootstrap': params['bootstrap']                 # Whether bootstrap samples are used when building trees
        }
        '''
        hype = {
            'criterion': params['criterion'],            # Criterion for splitting ('gini' or 'entropy')
            'max_depth': params['max_depth'],     # Maximum depth of the tree
            'min_samples_split': params['min_samples_split'],         # Minimum number of samples required to split an internal node
            'min_samples_leaf': params['min_samples_leaf'],             # Minimum number of samples required to be at a leaf node
            'splitter': params['splitter']
        }

        # Run the pipeline with the current hyperparameters
        met = pipeline(X, pos_y, hype, cross_split=cross_split, balance_split=balance_split, balance_ratio=balance_ratio, classifier=classifier)
        score = float(met['roc_auc'][:-9])

        # Store the result
        print(f"trial: {trial}: score: {score}")
        results.append({'params': params, 'score': score})

    # Sort results by score in descending order
    results = sorted(results, key=lambda x: x['score'], reverse=True)

    best_params = results[0]['params']
    best_score = results[0]['score']

    return best_params, best_score, results

In [21]:
pos_y2 = r'/Users/georgeilli/Desktop/NeurIPS/NeurIPS-2024-GANTAT/Data/Labels/pancreatic intraductal papillary-mucinous neoplasm-Tclin.csv'
pos_y3 = r'/Users/georgeilli/Desktop/NeurIPS/NeurIPS-2024-GANTAT/Data/Labels/acute myeloid leukemia-tclin.csv'
pos_y = r'/Users/georgeilli/Desktop/NeurIPS/NeurIPS-2024-GANTAT/Data/Labels/Tclin.csv'


graph_ = r"/Users/georgeilli/Desktop/NeurIPS/NeurIPS-2024-GANTAT/Network Construction/less_Tclin_Signalink_PIN_graph.graphml"
graph = ig.Graph.Load(graph_, format='graphml')

data = []
for vertex in graph.vs:
    # Get node attributes excluding 'label'
    node_data = {key: vertex[key] for key in vertex.attributes() if key not in ['label', 'id']}
    
    # Ensure 'name' is the first key
    if 'name' in node_data:
        node_data = {'Gene_name': node_data.pop('name'), **node_data}
    
    data.append(node_data)

# Convert to DataFrame
X = pd.DataFrame(data)
X_ = pd.concat([X, embedding_df.iloc[:, 1:-1]], axis=1)
X_.columns = [X_.columns[0]] + list(range(1, len(X_.columns)))
X_


,Gene_name,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,A1BG,14.0,15.0,6.0,0.0,3.0,4.0,0.0,0.0,0.0,...,-0.431591,0.060424,-0.129183,-0.481415,-0.568865,-0.303235,0.381311,0.329220,-0.489454,0.359802
1,A2M,36.0,45.0,31.0,3.0,16.0,13.0,2.0,4.0,0.0,...,-0.624099,-0.378534,0.587614,0.204790,-0.001708,-0.260322,0.163261,0.009044,-0.046713,-0.188634
2,AAAS,4.0,11.0,17.0,0.0,0.0,1.0,0.0,5.0,1.0,...,-0.235663,0.228142,0.528476,-0.797465,-0.295607,0.109942,-0.239211,-0.266232,-0.596889,-0.127354
3,AAMP,19.0,9.0,6.0,2.0,4.0,5.0,1.0,4.0,1.0,...,-1.454517,-0.296607,0.266557,-0.412165,0.080035,0.305463,0.500402,-0.632157,-0.740676,0.444917
4,AANAT,1.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,-0.627908,0.250083,0.204159,-0.498247,-0.012303,0.681490,-0.435067,-0.805152,-0.378696,0.023666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,ZSCAN22,5.0,5.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,...,-0.384159,0.009529,0.163408,0.178857,0.010014,-0.087363,-0.076601,-0.248210,-0.068387,-0.247631
6082,ZW10,8.0,11.0,0.0,0.0,3.0,2.0,1.0,0.0,0.0,...,0.263163,0.247511,0.286515,0.027502,0.433272,0.034803,0.501197,-0.145684,-0.726008,-0.325477
6083,ZXDC,15.0,34.0,26.0,1.0,4.0,1.0,0.0,7.0,0.0,...,-0.529870,-0.317962,0.550444,0.194613,-0.014248,-0.102319,-0.024417,0.019715,-0.304211,0.616188
6084,ZYG11B,14.0,17.0,12.0,1.0,2.0,2.0,1.0,0.0,0.0,...,-0.365449,0.025736,-0.018257,-0.068455,-0.539451,-0.246646,0.166998,-0.452764,-0.295334,-0.118572


In [30]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import itertools
import xgboost as xgb

param_grid_xgb = {
    'learning_rate': [0.01, 0.05, 0.1],  # Different learning rates
    'max_depth': [10],               # Different depths of trees
    'n_estimators': [300],           # Number of trees
    'subsample': [0.6, 1.0],             # Fraction of samples used for fitting
    'colsample_bytree': [0.6, 1.0]       # Fraction of features used for fitting
}

param_grid_dt = {
    'criterion': ['entropy'],            # Criterion for splitting ('gini' or 'entropy')
    'max_depth': [3, 7, 10, 12],     # Maximum depth of the tree
    'min_samples_split': [2, 5, 10, 20, 30],         # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 8, 12],             # Minimum number of samples required to be at a leaf node
    'splitter': ['best']
}

param_grid_rf = {
    'n_estimators': [300],            # Number of trees in the forest
    'max_depth': [15],            # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],           # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],             # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt'],    # Number of features to consider when looking for the best split
    'bootstrap': [False]                 # Whether bootstrap samples are used when building trees
}
param, score, log = grid_search(X_, pos_y3, param_grid_dt, cross_split=5, balance_split=30, balance_ratio=2, classifier='DT')

param, score

accuracy: 0.8191 ± 0.0151
precision: 0.0256 ± 0.0149
recall: 0.5720 ± 0.3335
f1: 0.0488 ± 0.0280
roc_auc: 0.7955 ± 0.0858
aupr: 0.1895 ± 0.1628
trial: 1: score: 0.7955
accuracy: 0.8151 ± 0.0200
precision: 0.0249 ± 0.0141
recall: 0.5720 ± 0.3335
f1: 0.0475 ± 0.0268
roc_auc: 0.7962 ± 0.0853
aupr: 0.1899 ± 0.1627
trial: 2: score: 0.7962
accuracy: 0.8212 ± 0.0268
precision: 0.0356 ± 0.0095
recall: 0.7636 ± 0.0593
f1: 0.0678 ± 0.0174
roc_auc: 0.8409 ± 0.0713
aupr: 0.2730 ± 0.1092
trial: 3: score: 0.8409
accuracy: 0.8616 ± 0.0606
precision: 0.0495 ± 0.0181
recall: 0.7355 ± 0.1522
f1: 0.0916 ± 0.0321
roc_auc: 0.8724 ± 0.0575
aupr: 0.2416 ± 0.0894
trial: 4: score: 0.8724
accuracy: 0.8465 ± 0.0298
precision: 0.0379 ± 0.0168
recall: 0.7238 ± 0.3133
f1: 0.0718 ± 0.0316
roc_auc: 0.8658 ± 0.0364
aupr: 0.1970 ± 0.1098
trial: 5: score: 0.8658
accuracy: 0.8151 ± 0.0200
precision: 0.0249 ± 0.0141
recall: 0.5720 ± 0.3335
f1: 0.0475 ± 0.0268
roc_auc: 0.7962 ± 0.0853
aupr: 0.1899 ± 0.1627
trial: 6: score:

({'criterion': 'entropy',
  'max_depth': 3,
  'min_samples_split': 30,
  'min_samples_leaf': 8,
  'splitter': 'best'},
 0.8849)

In [117]:
params_xgb = {'learning_rate': 0.01,
  'max_depth': 10,
  'n_estimators': 300,
  'subsample': 1.0,
  'colsample_bytree': 0.6}

met = pipeline(X, pos_y, hype=params_xgb, cross_split=5, balance_split=30, balance_ratio=2, classifier='XgBoost')

met

accuracy: 0.9137 ± 0.0076
precision: 0.4139 ± 0.0314
recall: 0.8432 ± 0.0211
f1: 0.5543 ± 0.0283
roc_auc: 0.9493 ± 0.0107
aupr: 0.5775 ± 0.0377


{'accuracy': '0.9137 ± 0.0076',
 'precision': '0.4139 ± 0.0314',
 'recall': '0.8432 ± 0.0211',
 'f1': '0.5543 ± 0.0283',
 'roc_auc': '0.9493 ± 0.0107',
 'aupr': '0.5775 ± 0.0377'}